In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

# Core column and expression functions
from pyspark.sql.functions import col, lit, when, lag, datediff

# Date-related functions
from pyspark.sql.functions import year, month

# Aggregation functions
from pyspark.sql.functions import sum as _sum, avg, count, rank

In [0]:
inpatient_silver = spark.read.parquet("/mnt/silver/inpatient/inpatient_silver")

Lenght of stay analysis

In [0]:
from pyspark.sql.functions import datediff

inpatient_los = inpatient_silver.withColumn(
    "length_of_stay",
    datediff(col("nch_bene_dschrg_dt"), col("clm_admsn_dt"))
).select(
    "desynpuf_id",
    "clm_id",
    "clm_admsn_dt",
    "nch_bene_dschrg_dt",
    "length_of_stay"
)

inpatient_los.write.mode("overwrite").parquet("/mnt/gold/inpatient/length_of_stay")


In [0]:
inpatient_los.show()

Top DRGs by frequency

In [0]:
from pyspark.sql.functions import count

top_drg = inpatient_silver.groupBy("clm_drg_cd") \
    .agg(count("clm_id").alias("claim_count")) \
    .orderBy("claim_count", ascending=False)

top_drg.write.mode("overwrite").parquet("/mnt/gold/inpatient/top_drg")


In [0]:
top_drg.show()

Admissions over time

In [0]:
admissions_by_date = inpatient_silver.groupBy("clm_admsn_dt") \
    .agg(count("clm_id").alias("admissions")) \
    .orderBy("clm_admsn_dt")

admissions_by_date.write.mode("overwrite").parquet("/mnt/gold/inpatient/admissions_by_date")


In [0]:
admissions_by_date.show()